# Introdução

*Script* que cria uma tabela que apresenta informações das Agências Ambientais da CETESB que atuam nos 645 município deo Estado de São Paulo.

In [1]:
import os
import time
import requests
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup

# Função

In [2]:
def get_AgenciaAmbiental(mun):
    # Município
    mun_url = mun.encode('ISO-8859-1')
    mun_url = urllib.parse.quote(mun_url)
    print('{} > {}'.format(mun, mun_url))
    
    # Set URL
    url = 'https://licenciamento.cetesb.sp.gov.br/municipioss.asp?muni={}'.format(mun_url)
    
    # Get Data
    r = requests.get(url)    
    soup = BeautifulSoup(r.content, 'html.parser')
    time.sleep(1)

    # Get all relevant data
    table = soup.find('table',  attrs={'width':'300'})
    rows = table.find_all('tr')
    
    # Create List Adress
    list_end = [mun]
    for row in rows:
        text = row.get_text()
        text = ' '.join(text.split())
        text = text.strip()
        list_end.append(text)
    
    # Rename List    
    for i in ['Bairro:', 'Cidade:', 'CEP:', 'Fone:', 'Fax:', 'E-mail:']:
        list_end = np.char.replace(list_end, i, '')
    list_end = [x.strip() for x in list_end]
    
    # Format Variables
    cep = '{:08d}'.format(int(list_end[5]))
    tel = list_end[6].strip().replace(' ', '').replace('-', ' ')
    fax = list_end[7].strip().replace(' ', '').replace('-', ' ')
    
    # Create Dictionary
    return {
        'municipio_cetesb' : list_end[0],
        'agencia'          : list_end[1],
        'endereco'         : list_end[2].split('nº')[0].strip(),
        'numero'           : list_end[2].split('nº')[1].strip(),
        'bairro'           : list_end[3],
        'municipio_sede'   : list_end[4].title(),
        'cep'              : '{}-{}'.format(cep[:5], cep[5:8]),
        'telefone'         : '({}) {}-{}'.format(tel[:2], tel[3:7], tel[7:]),
        'fax'              : '({}) {}-{}'.format(fax[:2], fax[3:7], fax[7:]),
        'email'            : list_end[8],
        'url'              : url
    }

Uma vez que a função foi definida, basta fazer um teste. Escolhi o município de Piracicaba, pois não tem acentos e grafias divergentes nas diferentes bases de dados.

In [3]:
get_AgenciaAmbiental('Piracicaba')

Piracicaba > Piracicaba


KeyboardInterrupt: 

# *Loop*

Há municípios que apresentam apóstrofo e muitas vezes é escrito como acento ou até mesmo divergência na grafia do nome do município, por exemplo, "Moji Mirim" e "Mogi Mirim". Por isso, optei por ler uma tabela que tem os nomes dos municípios utilizados pela CETESB e, uma vez que fiz isso, basta inserir o nome que a instituição padronizou, por meio de um *loop* (*for*).

In [ ]:
# Read Dataframe
df = pd.read_csv(
    os.path.join('data', 'tabs', 'tab_municipios.csv'),
)

# Results
df.head()

In [ ]:
# Create empty list
list_addresses = []

#df = df[0:4]  # For tests only
for index, row in df.iterrows():
    # Create Small Dictionary
    dict_df = {
        'id_municipio': row['id_municipio'],        
    }
    
    # Get Addresses
    list_address = get_AgenciaAmbiental(row['municipio_cetesb'])
    list_address = {**dict_df, **list_address}
    list_addresses.append(list_address)

# Tabela Municípios

In [339]:
# Create Table from dictionarys
df = pd.DataFrame.from_dict(list_addresses)

# Results
df.head()

,id_municipio,municipio_cetesb,agencia,endereco,numero,bairro,municipio_sede,cep,telefone,fax,email,url
0,3500105,ADAMANTINA,Agência Ambiental de Dracena,Rua Tomé de Souza,438,Centro,Dracena,17900-000,(18) 3822-3166,(18) 3822-3166,dracena_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
1,3500204,ADOLFO,Agência Ambiental de São José do Rio Preto,Av. Floriano André Cabrera,,Res.Cidade Jardim- J.S. Marcos,São José Do Rio Preto,15081-490,(17) 3218-4300,(17) 3218-4300,riopreto_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
2,3500303,AGUAÍ,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,345,Centro,São João Da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
3,3500402,ÁGUAS DA PRATA,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,345,Centro,São João Da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
4,3500501,ÁGUAS DE LINDÓIA,Agência Ambiental de Mogi Guaçu,Rua Hugo Pancieira,400,Pedregulhal,Mogi Guaçu,13845-190,(19) 3818-0396,(19) 3818-0396,mogiguacu_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...


In [340]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Results
df.head()

,id_municipio,municipio_nome,municipio_cetesb,agencia,endereco,numero,bairro,municipio_sede,cep,telefone,fax,email,url
0,3500105,Adamantina,ADAMANTINA,Agência Ambiental de Dracena,Rua Tomé de Souza,438,Centro,Dracena,17900-000,(18) 3822-3166,(18) 3822-3166,dracena_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
1,3500204,Adolfo,ADOLFO,Agência Ambiental de São José do Rio Preto,Av. Floriano André Cabrera,,Res.Cidade Jardim- J.S. Marcos,São José Do Rio Preto,15081-490,(17) 3218-4300,(17) 3218-4300,riopreto_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
2,3500303,Aguaí,AGUAÍ,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,345,Centro,São João Da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
3,3500402,Águas da Prata,ÁGUAS DA PRATA,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,345,Centro,São João Da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
4,3500501,Águas de Lindóia,ÁGUAS DE LINDÓIA,Agência Ambiental de Mogi Guaçu,Rua Hugo Pancieira,400,Pedregulhal,Mogi Guaçu,13845-190,(19) 3818-0396,(19) 3818-0396,mogiguacu_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...


In [341]:
# Split
df[['endereco','complemento']] = df['endereco'].str.split(' - ', expand=True)

# Reorder
cols = df.columns.tolist()
cols = cols[0:5] + [cols[-1]] + cols[5:-1]
df = df[cols]

In [342]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Nome
        'Av ': 'Av. ',
        'Av. ': 'Avenida ',
        ' Joao ': ' João ',
        'Ten.'    : 'Tenente',
        
        # Basics
        ' Com '  : ' com ',
        ' Sobre ': ' sobre ',
        ' Da '   : ' da ',
        ' De '   : ' de ',
        ' Do '   : ' do ',
        ' Das '  : ' das ',
        ' Dos '  : ' dos ',
        ' A '    : ' a ',
        ' As '   : ' as ',
        ' Ao '   : ' ao ',
        ' Aos '  : ' aos ',
        ' E '    : ' e ',
        ' O '    : ' o ',
        ' Os '   : ' os ',
        
        '(11) -' : '',
        '( ) -'  : '',
        
        
        'Jd. '  : 'Jardim ',
        'V. '   : 'Vila ',
        'Sta.': 'Santa',
        'Pq. '   : 'Parque ',
        'Res.': 'Residencial ',
        ' Sao ' : ' São ',
        'J.S. Marcos': 'Jardim São Marcos',
        'Jardim-': 'Jardim -',
        'Xxiii': 'XXIII',
        'Ii': 'II',

        # Empty
        'None' : '',
        'none' : '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [343]:
# Adjust Table
df.loc[:, 'endereco'] = df['endereco'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'bairro'] = df['bairro'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'municipio_sede'] = df['municipio_sede'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'fax'] = df['fax'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'complemento'] = df['complemento'].astype(str).apply(lambda x: rename_nome(x))

# Results
df.head()

,id_municipio,municipio_nome,municipio_cetesb,agencia,endereco,complemento,numero,bairro,municipio_sede,cep,telefone,fax,email,url
0,3500105,Adamantina,ADAMANTINA,Agência Ambiental de Dracena,Rua Tomé de Souza,,438,Centro,Dracena,17900-000,(18) 3822-3166,(18) 3822-3166,dracena_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
1,3500204,Adolfo,ADOLFO,Agência Ambiental de São José do Rio Preto,Avenida Floriano André Cabrera,,,Residencial Cidade Jardim - Jardim São Marcos,São José do Rio Preto,15081-490,(17) 3218-4300,(17) 3218-4300,riopreto_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
2,3500303,Aguaí,AGUAÍ,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,,345,Centro,São João da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
3,3500402,Águas da Prata,ÁGUAS DA PRATA,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,,345,Centro,São João da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
4,3500501,Águas de Lindóia,ÁGUAS DE LINDÓIA,Agência Ambiental de Mogi Guaçu,Rua Hugo Pancieira,,400,Pedregulhal,Mogi Guaçu,13845-190,(19) 3818-0396,(19) 3818-0396,mogiguacu_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...


In [344]:
#set(df['agencia'])
#set(df['endereco'])
#set(df['numero'])
#set(df['bairro'])
#set(df['municipio_sede'])
#set(df['cep'])
#set(df['telefone'])
#set(df['fax'])
#set(df['email'])

In [345]:
# Deleta colunas
df.drop(['municipio_cetesb'], axis=1, inplace=True)

# Results
df.head()

,id_municipio,municipio_nome,agencia,endereco,complemento,numero,bairro,municipio_sede,cep,telefone,fax,email,url
0,3500105,Adamantina,Agência Ambiental de Dracena,Rua Tomé de Souza,,438,Centro,Dracena,17900-000,(18) 3822-3166,(18) 3822-3166,dracena_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
1,3500204,Adolfo,Agência Ambiental de São José do Rio Preto,Avenida Floriano André Cabrera,,,Residencial Cidade Jardim - Jardim São Marcos,São José do Rio Preto,15081-490,(17) 3218-4300,(17) 3218-4300,riopreto_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
2,3500303,Aguaí,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,,345,Centro,São João da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
3,3500402,Águas da Prata,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,,345,Centro,São João da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
4,3500501,Águas de Lindóia,Agência Ambiental de Mogi Guaçu,Rua Hugo Pancieira,,400,Pedregulhal,Mogi Guaçu,13845-190,(19) 3818-0396,(19) 3818-0396,mogiguacu_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...


In [347]:
# Save
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_cetesb.csv'),
    index=False,
)

# Tabela CETESB

In [353]:
# Save
df = pd.read_csv(
    os.path.join('data', 'tabs', 'tab_municipio_cetesb.csv'),    
)
df.head()

,id_municipio,municipio_nome,agencia,endereco,complemento,numero,bairro,municipio_sede,cep,telefone,fax,email,url
0,3500105,Adamantina,Agência Ambiental de Dracena,Rua Tomé de Souza,NaN,438,Centro,Dracena,17900-000,(18) 3822-3166,(18) 3822-3166,dracena_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
1,3500204,Adolfo,Agência Ambiental de São José do Rio Preto,Avenida Floriano André Cabrera,NaN,NaN,Residencial Cidade Jardim - Jardim São Marcos,São José do Rio Preto,15081-490,(17) 3218-4300,(17) 3218-4300,riopreto_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
2,3500303,Aguaí,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,NaN,345,Centro,São João da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
3,3500402,Águas da Prata,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,NaN,345,Centro,São João da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
4,3500501,Águas de Lindóia,Agência Ambiental de Mogi Guaçu,Rua Hugo Pancieira,NaN,400,Pedregulhal,Mogi Guaçu,13845-190,(19) 3818-0396,(19) 3818-0396,mogiguacu_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...


In [354]:
# Get column's name
cols = list(df.columns[2:-1])

In [355]:
# Group
df = pd.DataFrame(df.groupby(columns, dropna=False)['id_municipio'].count())
df = df.reset_index()
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

# Results
df.head()

,agencia,endereco,numero,complemento,bairro,municipio_sede,cep,telefone,fax,email
0,Agência Ambiental de Americana,Rua Florindo Cibin,435,NaN,Vila Medon,Americana,13465-230,(19) 3461-1550,(19) 3461-1550,americana_cetesb@sp.gov.br
1,Agência Ambiental de Araraquara,Avenida Castro Alves,1271,NaN,Carmo,Araraquara,14800-140,(16) 3332-2211,(16) 3332-2211,araraquara_cetesb@sp.gov.br
2,Agência Ambiental de Araçatuba,Rua Tenente Alcides Teodoro dos Santos,260,NaN,Aviação,Araçatuba,16055-557,(18) 3608-8053,(18) 3608-7197,aracatuba_cetesb@sp.gov.br
3,Agência Ambiental de Assis,Via Chico Mendes,75,NaN,Quinta dos Flamboyants,Assis,19810-005,(18) 3324-4177,(18) 3324-4177,assis_cetesb@sp.gov.br
4,Agência Ambiental de Atibaia,Rua Castro Fafe,295,NaN,Centro,Atibaia,12940-440,(11) 4411-6466,(11) 4411-6466,atibaia_cetesb@sp.gov.br


In [356]:
# Save
df.to_csv(
    os.path.join('data', 'tabs', 'tab_cetesb.csv'),
    index=False,    
)

# Results
df.head()

,agencia,endereco,numero,complemento,bairro,municipio_sede,cep,telefone,fax,email
0,Agência Ambiental de Americana,Rua Florindo Cibin,435,NaN,Vila Medon,Americana,13465-230,(19) 3461-1550,(19) 3461-1550,americana_cetesb@sp.gov.br
1,Agência Ambiental de Araraquara,Avenida Castro Alves,1271,NaN,Carmo,Araraquara,14800-140,(16) 3332-2211,(16) 3332-2211,araraquara_cetesb@sp.gov.br
2,Agência Ambiental de Araçatuba,Rua Tenente Alcides Teodoro dos Santos,260,NaN,Aviação,Araçatuba,16055-557,(18) 3608-8053,(18) 3608-7197,aracatuba_cetesb@sp.gov.br
3,Agência Ambiental de Assis,Via Chico Mendes,75,NaN,Quinta dos Flamboyants,Assis,19810-005,(18) 3324-4177,(18) 3324-4177,assis_cetesb@sp.gov.br
4,Agência Ambiental de Atibaia,Rua Castro Fafe,295,NaN,Centro,Atibaia,12940-440,(11) 4411-6466,(11) 4411-6466,atibaia_cetesb@sp.gov.br
